In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, HashingTF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [2]:
import locale

locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

In [3]:
# create a SparkSession 
spark = SparkSession.builder \
    .appName("assignment 3") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.cores", "2") \
    .getOrCreate()

23/05/08 11:32:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# read csv

df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiline", "true") \
    .option("delimiter", "\t") \
    .csv("reviews.csv")

df = df.select(col('review_text'), col('label'))
df = df.dropna()
df.show()

+--------------------+-----+
|         review_text|label|
+--------------------+-----+
|i can confirm tha...|  1.0|
|Really good game,...|  1.0|
|Its not finished ...|  1.0|
|Hey. It's really ...|  1.0|
|          Fun so far|  1.0|
|I'd rather play W...|  0.0|
|I have been playi...|  1.0|
|Nice game! Loads ...|  1.0|
|All hail NA serve...|  1.0|
|enjoying it so fa...|  1.0|
|This game came as...|  1.0|
|         Great fun  |  1.0|
|Fantastic consept...|  1.0|
|You know the game...|  1.0|
|EDIT: My issues h...|  1.0|
|AMAZIIIIIIIIIIING...|  1.0|
|In its current st...|  0.0|
|Играю в Калибр го...|  1.0|
|Secret Word: Prou...|  1.0|
|Fated Word: Death...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [5]:
df.select("label").distinct().show()

+--------------------+
|               label|
+--------------------+
|                 1.0|
|                 0.0|
|It's been at leas...|
+--------------------+



In [6]:
df = df.filter((col("label") == 1.0) | (col("label") == 0.0))
print("Number of rows after filtering: ", df.count())

Number of rows after filtering:  18461


In [7]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=7)

In [8]:
train_data.show()

+--------------------+-----+
|         review_text|label|
+--------------------+-----+
|                    |  0.0|
|   I like trains,...|  1.0|
|  It's Free video...|  1.0|
|  Pros: 1. Feels ...|  1.0|
|  So I have put a...|  1.0|
| 75 minutes in i ...|  1.0|
|                  :)|  1.0|
| A very nice and ...|  1.0|
| Andrei Vasiu CAW...|  1.0|
| BEST GAME EVER, ...|  1.0|
| DO NOT BUY THIS ...|  0.0|
| Fun, challenging...|  1.0|
| Game has a lot o...|  1.0|
| Game sucks! The ...|  0.0|
| I actually put t...|  1.0|
| I honestly with ...|  1.0|
| I just started p...|  1.0|
| I like how they ...|  1.0|
| I love that game...|  1.0|
| I love this Game...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [9]:
# preprocess the data
tokenizer = Tokenizer(inputCol="review_text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", locale="en_US")
count_vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")
string_indexer = StringIndexer(inputCol="label", outputCol="label_index")

# create model
lr = LogisticRegression(featuresCol="features", labelCol="label_index")

23/05/08 11:32:45 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [10]:
# define params grid
param_grid = ParamGridBuilder() \
    .addGrid(count_vectorizer.vocabSize, [1000, 5000]) \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# define the evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label_index")


In [12]:
# create pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, string_indexer, lr])

# define the cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# fit pipeline to the training data
cv_model = cv.fit(train_data)

# make predictions on the test data
predictions = cv_model.transform(test_data)

23/05/08 11:33:12 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:33:12 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:33:19 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/08 11:33:19 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/05/08 11:33:19 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/08 11:33:19 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


23/05/08 11:33:21 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:33:23 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:33:27 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:33:28 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:33:37 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:33:38 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


...

23/05/08 11:37:06 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:06 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:37:10 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:10 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:14 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:15 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:37:19 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:19 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:23 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:23 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:37:26 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:27 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/08 11:37:29 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/08 11:37:30 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [13]:
# evaluate the model
accuracy = evaluator.evaluate(predictions)
print("Accuracy: ", accuracy)

Accuracy:  0.8847625878438992


In [14]:
from pyspark.mllib.evaluation import MulticlassMetrics

# Convert the predictions and labels to an RDD
predictionAndLabels = predictions.select("prediction", "label_index").rdd.map(lambda r: (r[0], r[1]))

# Instantiate a MulticlassMetrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Get the confusion matrix as a NumPy array
confusion_matrix = metrics.confusionMatrix().toArray()

# Print the confusion matrix
print("Confusion matrix:")
print(confusion_matrix)


/Users/anhphuong/Documents/KUL/Advanced Analytics/spark/spark-3.3.2-bin-hadoop2/python/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion matrix:
[[2979.   61.]
 [ 469.  235.]]
